In [ ]:
import streamlit as st
from joblib import load
import numpy as np
import pandas as pd


# Load the model
#@st.cache_data(allow_output_mutation=True)


# Load the trained model encoders, scalers, and label encoders
scaler = load('pipeline/scaler.joblib')
label_encoders = {
    'Zip Code': load('pipeline/Zip Code_label_encoder.joblib'),
    'Internet Service': load('pipeline/Internet Service_label_encoder.joblib'),
    'Online Security': load('pipeline/Online Security_label_encoder.joblib'),
    'Tech Support': load('pipeline/Tech Support_label_encoder.joblib'),
    'Contract': load('pipeline/Contract_label_encoder.joblib'),
    'Dependents': load('pipeline/Dependents_label_encoder.joblib'),
}

def preprocess_data(df):
    # Select relevant features
    features = ['Zip Code', 'Internet Service', 'Online Security', 'Tech Support', 'Contract', 'Dependents', 'Tenure Months', 'Monthly Charges', 'CLTV']
    df = df[features]

    # Handle missing values
    df.fillna(method='ffill', inplace=True)

    # Encode categorical features using LabelEncoders
    categorical_features = ['Internet Service', 'Online Security', 'Tech Support', 'Contract', 'Dependents', 'Zip Code']
    for feature in categorical_features:
        df[feature] = label_encoders[feature].transform(df[feature])
    
    # Scale numerical features
    numerical_features = ['Tenure Months', 'Monthly Charges', 'CLTV']
    df[numerical_features] = scaler.transform(df[numerical_features])

    return df

def load_model():
    model = load('pipeline/xgb.pkl')
    return model

model = load_model()

input_data = []
# Preprocess the input data
preprocessed_data = preprocess_data(input_data)
    
# Make prediction
prediction = model.predict(preprocessed_data)
churn_prob = model.predict_proba(preprocessed_data)[0][1]
print(f'The predicted churn status is: {"Churn" if prediction[0] == 1 else "No Churn"}')
print(f'The probability of churn is: {churn_prob:.2f}')       